### Feature Selection -


        This notebook is for implementing various ways for feature selection methods. Various different ways are :
        
        a. Pearson Correlation
        b. Chi Square using SelectKBest
        c. RFE
        d. SelectFrom Model- Logistic Regression, Random Forest & LGBM Classifier



In [27]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


import warnings

warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
 

In [2]:
df = pd.read_csv('../Feature Selection/Dataset/application_train.csv')
print('Number of rows', df.shape[0])
print('Number of columns',df.shape[1])

Number of rows 307511
Number of columns 122


In [3]:
('Statistics of the dataset')
df.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,...,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,...,0.008130,0.000595,0.000507,0.000335,0.006402,0.007000,0.034362,0.267395,0.265474,1.899974
std,102790.175348,0.272419,0.722121,2.371231e+05,4.024908e+05,14493.737315,3.694465e+05,0.013831,4363.988632,141275.766519,...,0.089798,0.024387,0.022518,0.018299,0.083849,0.110757,0.204685,0.916002,0.794056,1.869295
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,189145.500000,0.000000,0.000000,1.125000e+05,2.700000e+05,16524.000000,2.385000e+05,0.010006,-19682.000000,-2760.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,278202.000000,0.000000,0.000000,1.471500e+05,5.135310e+05,24903.000000,4.500000e+05,0.018850,-15750.000000,-1213.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,367142.500000,0.000000,1.000000,2.025000e+05,8.086500e+05,34596.000000,6.795000e+05,0.028663,-12413.000000,-289.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,456255.000000,1.000000,19.000000,1.170000e+08,4.050000e+06,258025.500000,4.050000e+06,0.072508,-7489.000000,365243.000000,...,1.000000,1.000000,1.000000,1.000000,4.000000,9.000000,8.000000,27.000000,261.000000,25.000000


### Datatypes

In [4]:
print('\n Number of datatypes in the dataset \n')

df.dtypes.value_counts()


 Number of datatypes in the dataset 



float64    65
int64      41
object     16
dtype: int64

### Stratified Sampling ratio:


    Stratified random sampling is a method of sampling that involves the division of a population into smaller groups known as strata. In stratified random sampling or stratification, the strata are formed based on members’ shared attributes or characteristics.

	Stratified random sampling is also called proportional random sampling or quota random sampling. The sample size of each stratum in this technique is proportionate to the population size of the stratum when viewed against the entire population.

In [5]:
df_sample1 = df.loc[df['TARGET']==1].sample(frac=0.1,replace=False)
df_sample0 = df.loc[df['TARGET']==0].sample(frac=0.1,replace=False)

print('size of Target 1 is',df_sample1.shape[0])
print('size of Target 0 is',df_sample0.shape[0])

application = pd.concat([df_sample0,df_sample1],axis=0,sort='SKI_ID_CURR')

print('New dataframe size is',application.shape)


size of Target 1 is 2482
size of Target 0 is 28269
New dataframe size is (30751, 122)


### Imputation Categorical values

In [6]:
from sklearn.impute import SimpleImputer

categ_list = []
num_list = []

for app in application.columns.to_list():
    if application[app].dtype=='object':
        categ_list.append(app)
    else:
        num_list.append(app)
        

application[num_list] = SimpleImputer(strategy='median').fit_transform(application[num_list])
application = pd.get_dummies(application,drop_first=True)

del df
print(application.shape)




(30751, 227)


### Feature list and Target value Selection

In [7]:
X = application.drop(['TARGET','SK_ID_CURR'],axis=1)
y = application.TARGET

feature_names = X.columns.tolist()



#### Feature Selection:
    
    
        Pearson Correlation: Without Normalization- Because by definition the correlation coefficient is independent of change of origin and scale. As such standardization will not alter the value of correlation.

In [8]:
def correlation_selector(X,y):
    
    corr_list = []
    
    for i in X.columns.tolist():
        cor = np.corrcoef(X[i],y)[0,1]
        corr_list.append(cor)
        
    #replace nan with 0
    
    corr_list = [0 if np.isnan(i) else i for i in corr_list]
    # Feature name
    cor_feature = X.iloc[:,np.argsort(np.abs(corr_list))[-100:]].columns.tolist()
    # Feature selection- 0 for not select, 1 for select
    cor_support = [True if i in cor_feature else False for i in feature_names]
    return cor_support, cor_feature

In [9]:
cor_support, cor_feature = correlation_selector(X, y)
print(str(len(cor_feature)), 'Selected features')

100 Selected features


#### Chi Square using SelectKBest

In [10]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import chi2,SelectKBest


X_norm = MinMaxScaler().fit_transform(X)
chi2_selector = SelectKBest(k=100).fit(X_norm,y)
chi_support = chi2_selector.get_support()
chi_feature = X.iloc[:,chi_support].columns.tolist()
print(len(chi_feature),'Selected features')



100 Selected features


### Wrapper


Feature Selection using RFE. 

The Recursive Feature Elimination (RFE) method is a feature selection approach. It works by recursively removing attributes and building a model on those attributes that remain. It uses the model accuracy to identify which attributes (and combination of attributes) contribute the most to predicting the target attribute.

In [11]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

rfe_selector = RFE(estimator=LogisticRegression(),n_features_to_select=100,step=10)
rfe_selector.fit(X_norm,y)

RFE(estimator=LogisticRegression(), n_features_to_select=100, step=10)

In [12]:
rfe_support = rfe_selector.get_support()
rfe_feature = X.loc[:,rfe_support].columns.tolist()
print(len(rfe_feature), 'Selected features')

100 Selected features


### Select From Model

In [16]:
from sklearn.feature_selection import SelectFromModel

lr_selector = SelectFromModel(LogisticRegression(),threshold='1.25*median')
lr_selector.fit(X_norm,y)

SelectFromModel(estimator=LogisticRegression(), threshold='1.25*median')

In [14]:
lr_support = lr_selector.get_support()
lr_feature = X.loc[:,lr_support].columns.tolist()
print(len(lr_feature),'Selected features')

98 Selected features


### RandomForest

In [19]:
from sklearn.ensemble import RandomForestClassifier

rf_selector = SelectFromModel(RandomForestClassifier(n_estimators=100),threshold='1.25*median').fit(X,y)
rf_support = rf_selector.get_support()
rf_feature = X.loc[:,rf_support].columns.tolist()
print(len(rf_feature),'Selected Features')



99 Selected Features


### Light GBM

In [21]:
from lightgbm import LGBMClassifier

lgb =LGBMClassifier(n_estimators=500, learning_rate=0.05, num_leaves=32, colsample_bytree=0.2,
            reg_alpha=3, reg_lambda=1, min_split_gain=0.01, min_child_weight=40)

lgb_selector = SelectFromModel(lgb,threshold='1.25*median').fit(X_norm,y)
lgb_support = lgb_selector.get_support()
lgb_feature = X.loc[:,lgb_support].columns.tolist()
print(len(lgb_feature),'Selected features')

110 Selected features


### Summary

In [26]:

feature_selection_df = pd.DataFrame({'Feature':feature_names, 'Pearson':cor_support, 'Chi-2':chi_support, 'RFE':rfe_support,
                                     'Logistics':lr_support,
                                    'Random Forest':rf_support, 'LightGBM':lgb_support})
# count the selected times for each feature
feature_selection_df['Total'] = np.sum(feature_selection_df, axis=1)
# display the top 100
feature_selection_df = feature_selection_df.sort_values(['Total','Feature'] , ascending=False)
feature_selection_df.index = range(1, len(feature_selection_df)+1)
feature_selection_df.head(100)

,Feature,Pearson,Chi-2,RFE,Logistics,Random Forest,LightGBM,Total
1,YEARS_BUILD_MEDI,True,True,True,True,True,True,6
2,YEARS_BUILD_AVG,True,True,True,True,True,True,6
3,REGION_RATING_CLIENT_W_CITY,True,True,True,True,True,True,6
4,OWN_CAR_AGE,True,True,True,True,True,True,6
5,NONLIVINGAREA_MODE,True,True,True,True,True,True,6
6,NAME_HOUSING_TYPE_Municipal apartment,True,True,True,True,True,True,6
7,NAME_HOUSING_TYPE_House / apartment,True,True,True,True,True,True,6
8,LIVINGAREA_MODE,True,True,True,True,True,True,6
9,LIVINGAREA_MEDI,True,True,True,True,True,True,6
10,LANDAREA_MEDI,True,True,True,True,True,True,6
